# Example: Launch Comfy templates on RunPod (safe demo)

This notebook provides a simple, team-friendly example to pick a GPU and a Comfy template and then fire a RunPod request from the `src.runpod` helper (which calls the `runpodctl` CLI).

Important safety notes:
- By default this notebook uses `DRY_RUN = True` so no actual pods will be created. Set `DRY_RUN = False` to perform real actions.
- Ensure your `.env` (or environment) exposes `RUNPOD_API_KEY` when you intend to perform real deployments.
- This example uses simple `input()` for choices so it works in classic and JupyterLab UIs without extra dependencies.

In [ ]:
import os
from getpass import getpass

# Safety flag: keep True by default to avoid accidental charges
DRY_RUN = True

# Optional: show whether RUNPOD_API_KEY is present (don't print the key)
has_key = 'RUNPOD_API_KEY' in os.environ and bool(os.environ.get('RUNPOD_API_KEY'))
print('RUNPOD_API_KEY present in environment:', has_key)
if not has_key:
    print('
Tip: to run real deployments set RUNPOD_API_KEY in the container environment or in a .env file mapped into the container.
')

# Import our Runpod helper client (installed editable in the devcontainer)
from src.runpod.client import RunpodClient

# Create client (will call `runpodctl` in the container). You can also pass runpodctl path: RunpodClient('/usr/local/bin/runpodctl')
rp = RunpodClient()

In [ ]:
# Define available templates and GPU options for the team
TEMPLATES = [
    'comfy-template',
    'comfy-vibeVoice',
]
GPU_CHOICES = ['3090', 'a10', 'v100', 'rtx6000']

def choose_from_list(prompt: str, options: list) -> str:
    print(prompt)
    for i, opt in enumerate(options, start=1):
        print(f'{i}) {opt}')
    while True:
        v = input('Select number (or press Enter for default 1): ').strip()
        if v == '':
            return options[0]
        try:
            idx = int(v) - 1
            if 0 <= idx < len(options):
                return options[idx]
        except Exception:
            pass
        print('Invalid selection, try again.')

In [ ]:
# Interactive selection (in a cell)
template = choose_from_list('Choose a template to launch:', TEMPLATES)
gpu = choose_from_list('Choose a GPU type:', GPU_CHOICES)
deployment_name = input('Deployment name (default: comfy-demo): ').strip() or 'comfy-demo'
scale = input('Scale (replicas) (default: 1): ').strip() or '1'
print(f'Chosen: template={template}, gpu={gpu}, name={deployment_name}, scale={scale}')

In [ ]:
# Favorite the template (idempotent) and create a deployment (dry-run default)
print('
--- Running favorite on template (dry-run={}) ---'.format(DRY_RUN))
rc, out, err = rp.favorite_template(template, dry_run=DRY_RUN)
print('favorite ->', rc, out if out else err)

print('
--- Creating deployment (dry-run={}) ---'.format(DRY_RUN))
rc2, out2, err2 = rp.create_deployment(template, gpu_type=gpu, name=deployment_name, extra_args=['--scale', str(scale)], dry_run=DRY_RUN)
print('create ->', rc2, out2 if out2 else err2)

if not DRY_RUN and rc2 == 0:
    print('Deployment creation returned success. If this is a real deployment, use the RunPod dashboard or `runpodctl deployment list` to view status.')
elif not DRY_RUN and rc2 != 0:
    print('Deployment creation failed; see stderr for details.')

---
Teardown notes:
- If you created a real deployment, remove it when finished with `runpodctl deployment delete <name>` or provide a small teardown cell here to call `rp.delete_deployment(name, dry_run=False)`.
- Consider adding cost controls (time-to-live) to deployments in `extra_args` for safer defaults.